# Biosound data - Tidy up 
Prep the biosound data for use in the dashboard

To run this in Pycharm you need to set up a kernel that points to the venv:
`python -m ipykernel install --user --name=venv-osa`

Also please note that the data files and folders in this notebook point to a specific path a local Google Drive shared folder. This will not work for others - the paths will need updating for each person.

In [1]:
import pandas as pd
import numpy as np

import data_wrangler as dw
import os

In [2]:
# Set up some initial things
# Local path to the main "Processed Data" folder
local_data_folder = ('/Users/michelle/Library/CloudStorage/GoogleDrive-michelle@waveformanalytics.com/.shortcut'
                     '-targets-by-id/1QNPk7Z3Yb2uhHsHCf49pw7k_2-ivhIJg/Processed_Data')

# config_file = os.path.join(local_data_folder, 'BioSound_Datasets_MapApp.csv')
config_file = os.path.join(local_data_folder, 'BioSound_Datasets_MapApp_v1.csv')
df_config = pd.read_csv(config_file)

annotations_folder = os.path.join(local_data_folder, 'Annotations')

# Load the fish/annotations codes lookup table
df_codes = pd.read_csv("../shiny/data/fish_codes.csv")

# Satellite water class data
seascaper_folder = "../shiny/shinydata/fromLiz/All_SeascapeR"


In [3]:
# Take a look at the contents of the config file
df_config

,Dataset,short name,Source,Contact,Sampling Rate (kHz),Bit Rate,Recording Cycle,Hydrophone Latitude,Hydrophone Longitude,Instrument Depth (m),Ecosystem Type,Selected Time Periods,Seascaper File,Annotations File
0,Caesar Creek,Biscayne Bay,University of Miami,Neil Hammerschlag/Abby Tinari,32,16,10 seconds every minute,25.396,-80.237,NaN,Mangrove,Feb - Mar 2019,CaesarCreek_SeascapeR_20190101_to_20190401.csv,NaN
1,Chuckchi Sea,Chukchi Sea,Oregon State University,Kate Stafford,16,16,25 minutes every hour,71.600,-161.500,30.0,Offshore,Jan - Feb 2019,ChuckChi_Sea_SeascapeR_20190101_to_20190401.csv,Chuckchi_Sea_Vessel_2019_02_ISO.csv
2,Gray's Reef,Gray's Reef,NEFSC-SanctSound,Tim Rowell,48,16,Continuous five hour files,31.396,-80.890,16.0,Offshore,Jan - Feb 2019,GraysReef_SeascapeR_20190101_to_20190401.csv,sanctsound_products_detections_gr01_sanctsound...
3,Key West,Key West,Florida Fish & Wildlife,Jess Keller,48,16,30 seconds every five minutes,24.442,-81.934,23.0,Coral Reef,Feb - Mar 2020,KeyWest_SeascapeR_20200101_to_20200401.csv,KW_Annotations.csv
4,May River,May River,University of S. Carolina Beaufort,Alyssa Marian/Eric Montie,80,16,2 minutes every hour,32.195,-80.792,4.5,Estuary,Jan - Mar 2019,MayRiver_SeascapeR_20190101_to_20190401.csv,MR_Annotations.csv
5,SanctSound-HI01,Olowalu,SanctSound - Hawaiian Islands Humpback NMS,Eden Zang,48,16,Continuous five hour files,20.807,-156.655,59.7,Island/ Nearshore,Jan - Feb 2019,HI01_NMS_SeascapeR_20190101_to_20190401.csv,sanctsound_products_detections_hi01_sanctsound...
6,ONC-MEF,ONC,Ocean Networks Canada,Jasper Kanes,64,24,Continuous 5 minute files,47.949,-129.098,2189.0,Offshore,Jan - Feb 2019,ONC_MEF_SeascapeR_20190101_to_20190401.csv,ONC_MEF_Vessels_2019_02_ISO.csv
7,OOI-HYDBBA106,OOI,Ocean Observatories Initiative,Liz Ferguson,64,16,Continuous 5 minute files,44.637,-124.306,80.0,Oregon Shelf,Jan - Feb 2019,HYDBBA106_SeascapeR_20190101_to_20190401.csv,OOI_HYDBBA105_Vessel_2019_02_ISO.csv


## Annotations data

### Prep Key West annotations
Key west annotations are a bit different from the others. They're in a folder that contains several .txt files that need to first be merged and then copied into the same annotations folder as the other datasets.

In [4]:
KW_ANNOTATIONS_FOLDER = os.path.join(local_data_folder, 'Annotations/key-west-original')
KW_ANNOTATIONS_OUTFILE = os.path.join(local_data_folder, 'Annotations/KW_Annotations.csv')

# Run the function from data_wrangler to convert to the regular annotations format and move the file to the same location as the rest of the annotation files
dw.annotation_prep_kw_style(KW_ANNOTATIONS_FOLDER, KW_ANNOTATIONS_OUTFILE)

### Prep May River annotations data

In [5]:
MR_ANNOTATIONS_FILE = os.path.join(local_data_folder, 'Annotations/may-river-original/Master_Manual_14M_2h_011119_071619.xlsx')
MR_ANNOTATIONS_OUTFILE = os.path.join(local_data_folder, 'Annotations/MR_Annotations.csv')

dw.annotation_prep_mr_style(MR_ANNOTATIONS_FILE, MR_ANNOTATIONS_OUTFILE, df_codes)

## Index data

In [6]:
# INDEX_DATA_FOLDER = os.path.join(local_data_folder, 'Revised_Indices_AllData_v2')
INDEX_DATA_FOLDER = os.path.join(local_data_folder, 'Original_Indices_AllData_v1')

df_aco0 = dw.prep_index_data(INDEX_DATA_FOLDER, normalize=False)
df_aco_norm0 = dw.normalize_df(df_aco0, df_aco0.columns[7:-2])

## Add the annotation information to index dataframes

In [7]:
unique_codes = np.unique(df_codes['code']).tolist()

# Add new columns that are named using unique_fish_codes. These will become the 
df_aco = dw.add_new_columns(df_aco0, unique_codes)
df_aco_norm = dw.add_new_columns(df_aco_norm0, unique_codes)

In [10]:
df_aco_anno = dw.add_annotations_to_df(df_aco, df_config, df_codes, annotations_folder)
df_aco_norm_anno = dw.add_annotations_to_df(df_aco_norm, df_config, df_codes, annotations_folder)